Import Tensorflow.

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import numpy as np
import pathlib

Make sure we use GPU for training.

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Create training and validation sets.

In [3]:
image_size = (100, 100)
batch_size = 32
epochs = 10
dataset_root_dir = pathlib.Path().absolute() / "../fruits-360_dataset/fruits-360/Training/"

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2,
    # horizontal_flip=True,
    # vertical_flip=True,
    # rotation_range=90,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # brightness_range=(0.7, 1.3),
    # zoom_range=(0.7, 1.3)
    )

train_ds = datagen.flow_from_directory(
    directory=dataset_root_dir,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset="training"
)

val_ds = datagen.flow_from_directory(
    directory=dataset_root_dir,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset="validation"
)


Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.


Define model.

In [4]:
def build_model(num_classes):
  model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=131
  )

  # Freeze the pretrained weights
  model.trainable = False

  # Rebuild top
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.2
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

  # Compile
  model = tf.keras.Model(model.input, outputs, name="EfficiencyNet")
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
  model.compile(
      optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
  )
  return model

model = build_model(131)

16719872/16705208 [==============================] - 1s 0us/step


In [5]:
checkpoint_filepath = pathlib.Path() / '../tmp/checkpoints/' / 'effnet_classifier.h5'
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

h2 = model.fit(
    train_ds, epochs=epochs, validation_data=val_ds, callbacks=[checkpoint_callback]
)

Epoch 1/10


c:\Users\Filip\Documents\Dev\School\bacherol-thesis\venv\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
c:\Users\Filip\Documents\Dev\School\bacherol-thesis\venv\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1694/1694 [==============================] - 285s 161ms/step - loss: 0.2157 - accuracy: 0.9569 - val_loss: 0.0488 - val_accuracy: 0.9862
Epoch 2/10
1694/1694 [==============================] - 271s 160ms/step - loss: 0.0187 - accuracy: 0.9956 - val_loss: 0.0368 - val_accuracy: 0.9892
Epoch 3/10
1694/1694 [==============================] - 272s 161ms/step - loss: 0.0133 - accuracy: 0.9964 - val_loss: 0.0528 - val_accuracy: 0.9841
Epoch 4/10
1694/1694 [==============================] - 266s 157ms/step - loss: 0.0114 - accuracy: 0.9965 - val_loss: 0.0411 - val_accuracy: 0.9884
Epoch 5/10
1694/1694 [==============================] - 267s 158ms/step - loss: 0.0092 - accuracy: 0.9971 - val_loss: 0.0416 - val_accuracy: 0.9867
Epoch 6/10
1694/1694 [==============================] - 269s 159ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.0446 - val_accuracy: 0.9859
Epoch 7/10
1694/1694 [==============================] - 267s 158ms/step - loss: 0.0084 - accuracy: 0.9975 - val_loss: 0.041

KeyboardInterrupt: 